# Examples of higher order functions and partial application

So far we have discussed some possibilities unlocked by higher order functions and partial application. Such a discussion is, in some sense, theoretical. Limiting one to only the theory, and then stating that "application is an excercise left to the reader" with an implication of triviality is inadequate. While it is certainly true that the theory lights the way and shows us the direction of what is indeed possible, it is the daily practice of the engineer that truly turns this potential into reality. Believing that this translation of the ideal into the pragmatic requires little more than faithful knowledge of the theory is childishly naïve at best. 

For this reason, we will now focus on building simple but concrete programs with the aforementioned techniques, in order to complement our first intuition with lots of examples.

<div class="alert alert-block alert-info">
Theory and practice together form a unit which cannot be easily dissolved. Theory defines general patterns, whereas practice concerns itself with the many concrete instances that such patterns represent.

There tends to be much more practice than theory. Take _functions_ as an example. The theory will simply state that functions exist, are defined and work in a given way. The practice of functions is much more complex: we must learn to interpret a real-world phenomenon in terms of functions, define the function itself, give it a name, etc. Moreover, while there exists a limited number (for the sake of this discussion let us say: *just one*) of useful definitions of the concept of functions, at the same time there exist infinite many concrete functions in practice that we might "type in code". Some of these are good and reasonable code, others are good code but very slow to run, others are bad code but very fast to run, other function implementations are intuitive, and so on.

The apparent chaos inherent to the concrete application of the practice is what leads us to building something of value and of use. Theory brings order to this chaos so that we can see underlying patterns and more effectively work our way through to our goals.

Once again: **neither theory nor practice** can stand without the other.
</div>

## A simple case

In the rest of this chapter we will define some recursive higher order functions. We will do so by using almost exclusively the generic combinators `repeat` and `then`. The choice of these combinators is not just the result of a random selection: `repeat` embodies the abstract concept of performing an action multiple times in sequence, whereas `then` chains two (potentially) different actions together into a new, more complex action.

Let us concretize our discussion by observing an example of a simple function which we are now extremely familiar with: the recursive function which draws a line.

In [1]:
def line(n):
    if n <= 0:
        return ""
    else:
        return "*" + line(n-1)
print(line(int(input())))

4


'****'

Let us focus now on the expression `"*" + line(n-1)` which characterizes the inductive case. We could _interpret_ this as _draw an asterisk, then the rest of the line_.

This way of interpreting code does not match the way that code would exactly be read, since we are not talking about arithmetic operations such as the sum of strings. This mismatch between reading and interpretation is not just merely annoying: it can be a source of dissonance between our expectations and what the code really does.

Our target is to be able to write code that literally matches the description: `then(asterisk, line(n-1))`. This would be read as `asterisk then line(n-1)` if we consider `then` to be a prefix operator, that is an operator which is written to the left of the arguments. Given our current state of the art, this is the best we can do, but moving forward with classes and methods it will be possible to improve on the readability even further.

There is a minor snag along the way: `then` does not work with strings, but rather with functions as parameters. This means that both `asterisk` and `line(n-1)` must return _functions_, which `then` will combine together, instead of just strings. This leads us to reformulating the task we expect of `line` so that it matches this new requirement:

**`line(n)` is a function which takes as input a string, and returns the input string, plus `n` asterisks attached to it**

We could also rephrase this by explicitly taking into account the first parameter to `line` as follows:

**`line` is a function which takes as input a number `n`, and returns a function which further takes as input a string, and returns the input string, plus `n` asterisks attached to it**

This means that `line(3)` does not directly return `"***"`. Rather, it returns a function which we can further invoke, with an extra set of parentheses with arguments between them. Thus, `line(3)("")` would yield `"***"`. Furthermore, `line(3)("...")` would give us `"...***"`, meaning that `line` simply "keeps drawing" from a given starting point. This has a major consequence: `line` is now ready to be concatenated to other drawing functions: `then(line(3), line(2))` would give us back a function which adds `"*****"` to whatever input it further receives. We can thus keep composing: `then(line(3), then(line(2), line(1)))` would give us back a function which adds `"******"` to whatever input it further receives, and so on.

According to this new, composable formulation of drawing:
- drawing nothing will simply add nothing to the string it receives as input
- drawing an asterisk will simply add an asterisk to the string it receives as input
- drawing a newline will simply add a newline to the string it receives as input    
- ...

We can turn this in code quite straightforwardly:

In [3]:
def nothing(s): return s
def asterisk(s): return s + "*"
def space(s): return s + " "
def newline(s): return s + "\n"

print(nothing(""))
print(nothing("..."))

print(asterisk(""))
print(asterisk("..."))


...
*
...*


This composability makes it easy, or at least facilitates, drawing multiple things in combination. For example, if we wanted to draw multiple asterisks in sequence, we could call the asterisk function nested within itself: the result of the first is then fed right into the second:

In [3]:
print(asterisk(asterisk("")))
print(asterisk(asterisk(asterisk(""))))

**
***


The example we have just given is a bit weak. It requires us to provide an initial string always, which might be something we want to delay for later. Moreover, invoking a function with the output of another one is exactly what we built `then` for, so we could now rephrase the code above in terms of `then`:

In [4]:
def then(f,g): 
    return lambda x: g(f(x))

two_asterisks = then(asterisk, asterisk)
three_asterisks = then(asterisk, two_asterisks)

print(two_asterisks(""))
print(three_asterisks(""))
print(three_asterisks("!!!"))
print(then(asterisk, then(space, asterisk))(""))

**
***
!!!***
* *


We have now shifted our focus from the very concrete idea of "concrete drawing", in which functions would directly output their drawing, to something more abstract. Our notion of "abstract drawing" delays the concrete operation, and simply accumulates instructions on _how to draw_. These instructions constitute a pipeline which, until it gets some initial string to draw from, performs no operation whatsoever.

Such a pipeline acts with the familiar rules for addition. For example, we could notice that just like `0 + n` is simply the same as `n`, `then(nothing, x)` will simply be the same as `x`. This basic fact suggests that drawing is and `then` to concatenate different images is indeed a "concatenative structure" just like we have seen strings to be:

In [5]:
print(then(nothing, asterisk)(""))
print(then(asterisk, nothing)(""))

*
*


At this point, we can "easily" assemble the `line` function:
- an empty line simply adds `nothing`, as there is indeed nothing to do;
- a non-empty line first adds an `asterisk`, `then` the rest of the `line`.

In [7]:
def line(n):
    if n <= 0:
        return nothing
    else:
        return then(asterisk, line(n-1))

line(int(input()))("")

4


'****'

### More complex figures

Let us move on to more complex figures. In particular, the first such figure we encounter when climbing up in complexity is the rectangle. According to the usual definition, a rectangle with `n` rows and `m` columns is drawn as:
- `nothing` if `n` is zero;
- a `line`, then a `newline`, and then a `rectangle` with n-1 rows otherwise.

With our framework, this is translated quite straightforwardly as:

In [8]:
def rectangle(n,m):
    if n <= 0:
        return nothing
    else:
        return then(then(line(m), newline), rectangle(n-1,m))
    
n = int(input())
print(rectangle(n,n)(""))

4
****
****
****
****



#### Repeating

We now start noticing a higher level pattern: we are indeed just repeating some (drawing) action multiple times. We might be (and indeed are!) tempted to capture this higher level pattern with an own name and as an own function.

This function is the also familiar `repeat`, which we could reformulate as a function which accepts a function as input, and then executes it `n` times starting from a given input.

`repeat(f,n)` will thus yield $\texttt{lambda x:} \underbrace{\texttt{f(...(f(x))...)}}_{n \text{ times}}$.

We can implement `repeat` now easily as a function which:
- returns `nothing` if `n` is zero;
- performs `f` once, `then` it `repeat`s `f` `n-1` times otherwise.

In [5]:
def repeat(f,n):
    if n <= 0:
        return nothing
    else:
        return then(f, repeat(f, n-1))

print(repeat(lambda x: x * 2, int(input()))(1))

4
16


We could even redefine a compacter and more elegant version of `line` by means of `repeat`:

In [8]:
def line(n): return repeat(asterisk, n)

print(line(int(input()))(""))

3
***


In the exact same spirit, a rectangle will just become the repetition of lines (each followed by a newline of course):

In [10]:
def rectangle(n,m): return repeat(then(line(m), newline), n)

print(rectangle(int(input()), int(input()))(""))

2
3
***
***



Finally, let us build hollow squares. Hollow squares are built up from hollow lines, but for the rest follow the same approach:

In [11]:
def hollow_line(n): return then(asterisk, then(repeat(space, n-2), asterisk))
def hollow_square(n): return then(then(line(n), newline), then(repeat(then(hollow_line(n), newline), n-2), line(n)))

print(hollow_square(int(input()))(""))

4
****
*  *
*  *
****


## Wrapping up
The approach we have just described falls under the broader name of _declarative programming_. Declarative programming is not a rigidly defined construct, and actually it is more of an umbrella concept which we use to categorize implementations of programs that follow the same general style.

Usually, a style of programming is defined as declarative if, instead of focusing on the update of variables in sequence in order to achieve a goal, we simply state the goal in code in a way that is read as almost natural language, and then via various means of composition the right answer is found. 

Declarative programming therefore allows us to quickly define solutions which are at a higher level of abstraction, and which therefore offer more chances for correctness than the imperative version of the same program. Working at a higher level of abstraction offers us the possibility to express our thoughts directly, unfiltered, in code, instead of performing a complex, cumbersome translation which might itself be a source of errors.